# Installing graph-tool (https://graph-tool.skewed.de)

---


To import a library that's not in Colaboratory by default, you can use `!apt-get install`.

But since graph-tool is not in the official repository, we need to add it to the list.

In [4]:
!echo "deb http://downloads.skewed.de/apt bionic main" >> /etc/apt/sources.list
!apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
!apt-get update
!apt-get install python3-graph-tool python3-matplotlib python3-cairo

zsh:1: no such file or directory: /etc/apt/sources.list
zsh:1: command not found: apt-key
zsh:1: command not found: apt-get
zsh:1: command not found: apt-get


In [5]:
#python3-cairo from Ubuntu's reposity is linked with a different python version; we need to improvise
!apt purge python3-cairo
!apt install libcairo2-dev pkg-config python3-dev
!pip install --force-reinstall pycairo
!pip install zstandard

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

The operation couldn’t be completed. Unable to locate a Java Runtime that supports apt.
Please visit http://www.java.com for information on installing Java.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.7/340.7 KB 4.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pycairo: filename=pycairo-1.21.0-cp39-cp39-macosx_10_9_x86_64.whl size=132959 sha256=85597a32fea282fd69b322a0a84df931554558d84eb88aa0dac8285215e2c785
  Stored in directory: /Users/andrea/Library/Caches/pip/wheels/0f/cd/70/dce236cd2318a543f9d2a78fa2c6e7170c575c203734885d25
Successfully built pycairo
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command

# Repository of weighted graphs: https://networks.skewed.de/?tags=Weighted

In [6]:
import numpy as np
import pandas as pd
import graph_tool.all as gt
from google.colab import files

from google.colab import drive

import os.path
from os import path

def export_all_weightedGraphs_erodeCSV(model_names,download_file,store_on_gdrive,path_g_drive="weightedGraphs/",prefix_to_weight=dict()):
   #Exports in ERODE CSV format all weighted graphs in the list model_names. 
   #The CSV file is either stored on google drive, or downloaded, as required by the parameters (Google drive has precedence on downloading)
   #If the google drive option is chosen, the files will be stored in folder weightedGraphs, or different paths provided in parameter path_g_drive
   #  This should be a relative path to a folder (ending with a slash). E.g., if you pass "weightedGraphs/", we will store files in it in MyDrive/weightedGraphs/


  print('Processing and exporting models')
  print("",model_names)

  #Prepare gdrive
  gdrive_path="/content/drive/MyDrive/"+path_g_drive
  if store_on_gdrive:
    drive.mount('/content/drive')
    if not path.exists(gdrive_path):
      os.mkdir(gdrive_path)
    if not path.exists(gdrive_path+"/csv/"):
      os.mkdir(gdrive_path+"/csv/")

  #Export required weighted graps
  for model_name in model_names:
    weight_name='weight'
    for key in prefix_to_weight:
      if model_name.startswith(key):
        weight_name=prefix_to_weight[key]
        break
    export_weightedGraph_erodeCSV(model_name,download_file,store_on_gdrive,gdrive_path,weight_name)

  #Close gdrive
  if store_on_gdrive:
    drive.flush_and_unmount()

  print('All computations completed')

def export_weightedGraph_erodeCSV(model_name,download_file,store_on_gdrive,gdrive_path,weight_name):
  #load model
  print("PROCESSING:",model_name,"...")

  g = gt.collection.ns[model_name]

  model_name = model_name.replace("/","_")
  model_name = model_name.replace(" ","_")
  model_name = model_name.replace("-","_")
  model_name = model_name.replace("(","_")
  model_name = model_name.replace(")","_")

  #store unweighted edges in a dataframe, and add weights
  df = pd.DataFrame(data=g.get_edges(), columns=["source", "target"])
  df["source"]=df["source"]+1
  df["target"]=df["target"]+1
  
  if weight_name in g.edge_properties:
    df['weight']=g.edge_properties[weight_name].get_array()
  else:
    df['weight']=np.ones(len(df))
    print("BEWARE:",model_name,"DOES NOT HAVE WEIGHTS.\n WE SET ALL WEIGHTS TO ONE!")
    model_name=model_name+"_defaultWeights"
  
  n_nodes=g.num_vertices()
  df0 = pd.DataFrame([[n_nodes,n_nodes,n_nodes]], columns=["source", "target",'weight'])
  df=pd.concat([df0, df])
  df.reset_index(drop=True, inplace=True)
  #print(df)

  dfB = pd.DataFrame(data=np.zeros(n_nodes), columns=["B"])

    
  #compute file name of csv file
  csv_file_name="csv/"+model_name+'A.csv'
  csv_file_nameB="csv/"+model_name+'B.csv'
  erode_file_name="import_"+model_name+'.ode'
    
  if store_on_gdrive:
    csv_file_name  =gdrive_path+"csv/"+model_name+'A.csv'
    csv_file_nameB =gdrive_path+"csv/"+model_name+'B.csv'
    erode_file_name=gdrive_path+"import_"+model_name+'.ode'
    
  

  print("  ","Graph processed, and data structure created")

  #create csv file and download if required
  df.to_csv( csv_file_name ,index=False,header=False)
  dfB.to_csv(csv_file_nameB,index=False,header=False)

  # Create ERODE file to import the CSV ones
  with open(erode_file_name, 'w') as fp:
    fp.write('begin model import_'+model_name+"\n")
    fp.write(' importAffineSystem(fileIn="csv/'+model_name+'A.csv", BFile="csv/'+model_name+'B.csv", ICFile="csv/'+model_name+'B.csv")\n')
    fp.write(' write(fileOut="ODE/'+model_name+'._ode",format=ODE)\n')
    fp.write(' reduceBE(reducedFile="BE/'+model_name+'_BE._ode", csvFile="BE.csv")\n')
    fp.write(' reduceFE(reducedFile="FE/'+model_name+'_FE._ode", csvFile="FE.csv")\n')
    fp.write('end model\n')
    
  
  if download_file and not store_on_gdrive:
    files.download(csv_file_name)
    files.download(csv_file_nameB)
    files.download(erode_file_name)

  print("  ","All files exported")

    
  
#Choose the models to consider
# Just https://networks.skewed.de/net/blumenau_drug
#model_names=["blumenau_drug"]
#model_names=["cattle"]
model_names=["blumenau_drug","cattle","train_terrorists"]

#All weighted networks on 21/07/2022 from smallest number of nodes to largest#
# Taken from https://networks.skewed.de/?tags=Weighted after clicking on 'Nodes'
#model_names=[
#              "packet_delays","ambassador/1985_1989","ambassador/1990_1994","ambassador/1995_1999","ambassador/2000","ambassador/2001","ambassador/2002","ambassador/2003","ambassador/2004","ambassador/2005","ambassador/2006","ambassador/2007","ambassador/KINSHIP","ambassador/OPERATION_ID","ambassador/TIE_EXTINGUISH","ambassador/TIE_YEAR",
#              "rhesus_monkey","kangaroo",
#              "sp_baboons/observational","sp_baboons/sensor","bison","cattle","moreno_sheep","college_freshmen","hens",
#              "freshmen/t0","freshmen/t2","freshmen/t3","freshmen/t5","freshmen/t6",
#              "windsurfers","macaques","train_terrorists","highschool","blumenau_drug","lesmis","baseball/player-player","baseball/user-provider","sp_office","game_thrones",
#              "moviegalaxies/1","moviegalaxies/2","moviegalaxies/3","moviegalaxies/5",#moviegalaxies contains about 1000 models. We consider only 4
#              "foodweb_baywet",
#              "plant_pol_vazquez/Llao Llao","plant_pol_vazquez/Cerro Lopez","plant_pol_vazquez/Safariland","plant_pol_vazquez/Arroyo Goye","plant_pol_vazquez/Mascardi (nc)","plant_pol_vazquez/Mascardi (c)","plant_pol_vazquez/Quetrihue (nc)","plant_pol_vazquez/Quetrihue (c)","plant_pol_vazquez/All sites pooled",
#              "dom/Adcock_2015a","dom/Adcock_2015b","dom/Alados_1992a","dom/Alados_1992b","dom/Alados_1992c","dom/Alados_1992d","dom/Alados_1992e","dom/Allee_1954","dom/Anderson_2016a","dom/Anderson_2016d","dom/Appleby_1983","dom/Archie_2006a","dom/Archie_2006b","dom/Archie_2006c","dom/Archie_2006d","dom/Archie_2006e","dom/Archie_2006f","dom/Archie_2006g","dom/Archie_2006h","dom/Archie_2006i","dom/Arlet_2015f","dom/Barette_1986","dom/Bennett_1939","dom/BergstromFedigan_2010b","dom/BergstromFedigan_2010c","dom/Berman_2004a","dom/Berman_2004b","dom/Berman_2004c","dom/Blatrix_2004b","dom/Blatrix_2004c","dom/Blatrix_2004d","dom/Blatrix_2004e","dom/Bonanni_2007a","dom/Bonanni_2007b","dom/Bonanni_2007c","dom/Bonanni_2017a","dom/Bonanni_2017b","dom/Bonanni_2017c","dom/Bonanni_2017d","dom/Bromley_1991","dom/Cafazzo_2010a","dom/Cafazzo_2010b","dom/Cheney_1977","dom/Cheney_1977b","dom/CluttonBrock_1976","dom/Collias_1950a","dom/Collias_1950b","dom/Collias_1950c","dom/Collias_1951a","dom/Collias_1951b","dom/Collias_1951c","dom/Correa_2013a","dom/Correa_2013d","dom/Cote_2000","dom/Cote_2000b","dom/Cote_2000c","dom/Cote_2000d","dom/Cui_2014","dom/David_2003a","dom/David_2003b","dom/DeLaFuente_2019","dom/DeLaO_2019a","dom/DeLaO_2019b","dom/DeLaO_2019c","dom/DeLaO_2019d","dom/deVries_2006","dom/deWaal_1977a","dom/deWaal_1977b","dom/deWaal_1985","dom/Diniz_2019","dom/Douglis_1948a","dom/Douglis_1948b","dom/Dubosq2013a","dom/Dubosq2013b","dom/Dubosq2013c","dom/Dubosq2013d","dom/Dubosq2013e","dom/Dubosq2013f","dom/Ellard_1989","dom/Essler_2016a","dom/Essler_2016c","dom/Essler_2016d","dom/Fairbanks_1994a","dom/Fairbanks_1994b","dom/Fairbanks_1994c","dom/Fairbanks_1994d","dom/Fairbanks_1994e","dom/Farentinos_1972a","dom/Farentinos_1972b","dom/Farentinos_1972c","dom/Farentinos_1972d","dom/Farentinos_1972e","dom/Farentinos_1972f","dom/Foerster_2016a","dom/Foerster_2016b","dom/Fournier_1995","dom/Fournier_1995b","dom/Frank_1986a","dom/Frank_1986b","dom/Franz_2015a","dom/Franz_2015b","dom/Franz_2015c","dom/Franz_2015d","dom/Franz_2015e","dom/Funkhouser_2018a","dom/Funkhouser_2018b","dom/Harcourt_1979","dom/Harcourt_1989","dom/Hartzler_1970","dom/Hass_1991","dom/Hass_1991b","dom/Hass_1991c","dom/Hausfater_1975a","dom/Hausfater_1975b","dom/Hausfater_1975c","dom/Hausfater_1982a","dom/Hausfater_1982b","dom/Hayaki_1989a","dom/Hayaki_1989b","dom/Heitor_2006","dom/HeitorVicente_2010","dom/Hewitt_2009a","dom/Hewitt_2009b","dom/Hewitt_2009c","dom/Hewitt_2009d","dom/Hirotani_1994","dom/Hirsch_2007a","dom/Hirsch_2007b","dom/Hirsch_2007c","dom/Hirsch_2007d","dom/Hobson_2015a","dom/Hobson_2015b","dom/Holekamp_1991","dom/Holekamp_1993a","dom/Holekamp_1993b","dom/Isbell_1998a","dom/Isbell_1998b","dom/Ito_1993a","dom/Ito_1993b","dom/Ito_1993c","dom/Izar_2006","dom/IzawaWatanabe_2011","dom/Jenks_1995a","dom/Jenks_1995b","dom/Johnson_2017","dom/Jones_1980a","dom/Jones_1980b","dom/Kaufmann_1974","dom/Kikkawa_1980","dom/Koenig_2004","dom/Kohda_1991","dom/Kolodziejczyk_2005","dom/Korstjens_2002","dom/Koutnik_1981","dom/Koutnik_1981b","dom/Kurvers_2009","dom/Lahti_1994a","dom/Lahti_1994b","dom/Langbein_2004a","dom/Langbein_2004b","dom/Langbein_2004c","dom/Langbein_2004d","dom/Langley_2018","dom/Lee_1979a","dom/Lee_1979b","dom/Lee_2017a","dom/Lee_2017b","dom/Lee_2017c","dom/Lee_2017d","dom/Lee_2017e","dom/Lee_2017f","dom/Lee_2018a","dom/Lee_2018b","dom/Lee_2018c","dom/Lee_2018d","dom/Lee_2018e","dom/Lee_2019a","dom/Lee_2019b","dom/Lee_2019c","dom/Lee_2019d","dom/Lee_2019e","dom/Lott_1979","dom/Lott_1987","dom/Lu_2008a","dom/Lu_2008b","dom/Malherbe_2007","dom/Marler_1955","dom/Masure_1934","dom/Masure_1934a","dom/Masure_1934b","dom/Mather_1985","dom/Matsuda_2012","dom/McCune_2019a","dom/McCune_2019c","dom/McCune_2019d","dom/McCune_2019e","dom/McCune_2019f","dom/McDougall_2010","dom/McMahan_1984","dom/Miller_2010a","dom/Miller_2010b","dom/Miller_2010c","dom/Moller_1987a","dom/Moller_1987b","dom/Moller_1987c","dom/Monnin_1999","dom/Moore_1978a","dom/Moore_1978b","dom/Murray_2007","dom/Mwamende_2009a","dom/Mwamende_2009b","dom/Myrberg_1972","dom/Myrberg_1974","dom/Nakano_1994","dom/Nakano_1995","dom/Natoli_1991","dom/Nelissen_1985a","dom/Nelissen_1985b","dom/NewtonFisher_2004a","dom/NewtonFisher_2004b","dom/Norscia_2015a","dom/Norscia_2015b","dom/Norscia_2015c","dom/Norscia_2015d","dom/Norscia_2015e","dom/Norscia_2015f","dom/Norscia_2015g","dom/Norscia_2015h","dom/Norscia_2015i","dom/Norscia_2015j","dom/Ortius_1995","dom/Oshea_1976","dom/Owens_1996","dom/Painter_2018","dom/Paoli_2006","dom/Pardi_1948a","dom/Pardi_1948b","dom/Parsons_1980a","dom/Parsons_1980b","dom/Patterson_1977","dom/Payne_2003","dom/Poisbleau_2005a","dom/Poisbleau_2005b","dom/Poisbleau_2005c","dom/Poisbleau_2006a","dom/Poisbleau_2006b","dom/Post_1992","dom/Prieto_1978","dom/Reason_1988","dom/Richter_2009","dom/RizaldiWatanabe_2010a","dom/RizaldiWatanabe_2010b","dom/Robbins_2008","dom/Roell_1978","dom/Rossler_2017","dom/Rovero_1999a","dom/Rovero_1999b","dom/Russell_1970a","dom/Russell_1970b","dom/Rutberg_1986a","dom/Rutberg_1986b","dom/Samuels_1987","dom/Samuels_1987b","dom/Sandel_2017a","dom/Sandel_2017b","dom/SatohOhkawara_2008a","dom/SatohOhkawara_2008b","dom/SatohOhkawara_2008c","dom/ScheinFohrman_1955","dom/SchjelderupEbbe_1922a","dom/SchjelderupEbbe_1922b","dom/ScottLockhard_1999a","dom/ScottLockhard_1999b","dom/Seibert_2001","dom/Setchell_2005a","dom/Setchell_2005b","dom/SetiaVanSchaik_2007","dom/Seyfarth_1976a","dom/Seyfarth_1976b","dom/Seyfarth_1976c","dom/Seyfarth_1976d","dom/Seyfarth_1976e","dom/Seyfarth_1976f","dom/Sharpe_2013a","dom/Sharpe_2013b","dom/Sharpe_2013c","dom/Sharpe_2013d","dom/Shimoji_2014a","dom/Shimoji_2014b","dom/Shimoji_2014c","dom/Shoemaker_1939","dom/Silk_2019a","dom/Slotow_1993","dom/Smith_1976","dom/Smith_2011a","dom/Solberg_1997a","dom/Solberg_1997b","dom/Solberg_1997c","dom/SomersNel_1998a","dom/SomersNel_1998b","dom/SomersNel_1998c","dom/Stamps_1978","dom/Sterck_1997","dom/Strauss_2019a","dom/Strauss_2019b","dom/Strauss_2019c","dom/Strauss_2019d","dom/StrayerCummins_1980","dom/StrayerCummins_1980b","dom/StrayerCummins_1980c","dom/StrayerCummins_1980d","dom/Struhsaker_1967a","dom/Struhsaker_1967b","dom/Sullivan_1982","dom/Surbeck_2011","dom/SurbeckHohmann_2013b","dom/Tamm_1977","dom/Tamura_1988","dom/Tarvin_1997","dom/Thompson_1960a","dom/Thompson_1960b","dom/Thompson_1960c","dom/Thompson_1960d","dom/Thompson_1960e","dom/Thompson_1993a","dom/Thompson_1993b","dom/Tilson_1984","dom/Tong_2020a","dom/Tong_2020b","dom/Tong_2020c","dom/Tong_2020d","dom/Tong_2020e","dom/Tong_2020f","dom/Tong_2020g","dom/Tong_2020h","dom/Tong_2020i","dom/Tong_2020j","dom/Torr_1996a","dom/Torr_1996b","dom/Torr_1996c","dom/Trebouet_2019a","dom/Trebouet_2019b","dom/Trebouet_2019c","dom/Trebouet_2019d","dom/Trunzer_1999","dom/Utt_2008a","dom/ValLaillet_2008a","dom/ValLaillet_2008b","dom/ValLaillet_2008c","dom/ValLaillet_2008d","dom/ValLaillet_2008e","dom/ValLaillet_2008f","dom/VanDessel_2018a","dom/VanDessel_2018b","dom/VanDierendonck_1995","dom/VanHooffWensing_1987","dom/Varley_1966","dom/Varley_1966b","dom/Vervaecke_2000","dom/Vervaecke_2010a","dom/Vervaecke_2010b","dom/Vervaecke_2010c","dom/Vervaecke_2010d","dom/Vilette_2020","dom/Waterhouse_1976a","dom/Waterhouse_1976b","dom/Waterhouse_1976c","dom/Watson_1970a","dom/Watson_1970b","dom/Watt_1986a","dom/Watt_1986b","dom/Watt_1986c","dom/Watt_1986d","dom/Watt_1986e","dom/Watt_1986f","dom/Watts_1985","dom/Watts_1994a","dom/Watts_1994b","dom/Watts_1994c","dom/Wells_1979","dom/WestEberhard_1986","dom/White_2007a","dom/White_2007b","dom/White_2007c","dom/Wikberg_2013a","dom/Wikberg_2013b","dom/Wikberg_2013c","dom/Wikberg_2013d","dom/Wikberg_2013e","dom/Wikberg_2013f","dom/Williams_1972","dom/Williamson_2016a","dom/Williamson_2016b","dom/Williamson_2016c","dom/Williamson_2016d","dom/Williamson_2016e","dom/Williamson_2016f","dom/Williamson_2016g","dom/Williamson_2016h","dom/Williamson_2016i","dom/Williamson_2016j","dom/Williamson_2016k","dom/Williamson_2017a","dom/Williamson_2017b","dom/Williamson_2017c","dom/Williamson_2017d","dom/Williamson_2017e","dom/Williamson_2017f","dom/Williamson_2017g","dom/Williamson_2017h","dom/Williamson_2017i","dom/Williamson_2017j","dom/Williamson_2017k","dom/Williamson_2017l","dom/Williamson_2017m","dom/Williamson_2017n","dom/Williamson_2017o","dom/Williamson_2019a","dom/Williamson_2019b","dom/Williamson_2019c","dom/Williamson_2019d","dom/Williamson_2019e","dom/Williamson_2019f","dom/Williamson_2019g","dom/Williamson_2019h","dom/Wittemeyer_2007","dom/Wittig_2003","dom/Yasukawa_1983a","dom/Yasukawa_1983b","dom/Yasukawa_1983c","dom/Yasukawa_1983d","dom/Yasukawa_1983e","dom/Yasukawa_1983f","dom/Yasukawa_1983g","dom/Yasukawa_1983h","dom/Zine_2000",
#              "cintestinalis",
#              "faculty_hiring/computer_science","faculty_hiring/business","faculty_hiring/history",
#              "fao_trade","residence_hall","un_migrations","sp_primary_school","celegansneural",
#              "sp_high_school/proximity","sp_high_school/diaries","sp_high_school/survey","sp_high_school/facebook",
#              "london_transport",
#              "sp_colocation/Thiers13","sp_colocation/SFHH","sp_colocation/LyonSchool","sp_colocation/LH10","sp_colocation/InVS15","sp_colocation/InVS13",
#              "webkb/webkb_wisconsin_link1","webkb/webkb_wisconsin_cocite","webkb/webkb_washington_link1","webkb/webkb_washington_cocite","webkb/webkb_cornell_link1","webkb/webkb_cornell_cocite","webkb/webkb_texas_link1","webkb/webkb_texas_cocite",
#             "celegans_2019/male_chemical","celegans_2019/male_gap_junction","celegans_2019/hermaphrodite_chemical","celegans_2019/hermaphrodite_gap_junction","celegans_2019/male_chemical_synapse","celegans_2019/male_gap_junction_synapse","celegans_2019/hermaphrodite_chemical_synapse","celegans_2019/hermaphrodite_gap_junction_synapse","celegans_2019/male_chemical_corrected","celegans_2019/male_gap_junction_corrected","celegans_2019/hermaphrodite_chemical_corrected","celegans_2019/hermaphrodite_gap_junction_corrected",
#             #"messal_shale", #DOES NOT WORK
#              "plant_pol_kato",
#              "copenhagen/sms","copenhagen/fb_friends","copenhagen/calls","copenhagen/bt",
#             "product_space/HS","product_space/SITC",
#             "unicodelang",
#             "budapest_connectome/all_20k","budapest_connectome/all_200k","budapest_connectome/all_1m","budapest_connectome/female_20k","budapest_connectome/female_200k","budapest_connectome/female_1m","budapest_connectome/male_20k","budapest_connectome/male_200k","budapest_connectome/male_1m",
#             "new_zealand_collab","netscience","bible_nouns",
#             # "add_health/comm1","add_health/comm2","add_health/comm3","add_health/comm4",#there are about 100 graphs. we consider only 4. BUT IT DOES NOT WORK: files offline
#             "openflights","bitcoin_alpha","bitcoin_trust","advogato","foldoc",
#             "physics_collab/pierreAuger","physics_collab/arXiv",
#             "escorts","fly_hemibrain",
#             "arxiv_collab/cond-mat-1999","arxiv_collab/cond-mat-2003","arxiv_collab/cond-mat-2005","arxiv_collab/astro-ph-1999","arxiv_collab/hep-th-1999",
#             "us_agencies/alabama","us_agencies/alaska","us_agencies/arizona","us_agencies/arkansas","us_agencies/california","us_agencies/colorado","us_agencies/connecticut","us_agencies/delaware","us_agencies/florida","us_agencies/georgia","us_agencies/hawaii","us_agencies/idaho","us_agencies/illinois","us_agencies/indiana","us_agencies/iowa","us_agencies/kansas","us_agencies/kentucky","us_agencies/louisiana","us_agencies/maine","us_agencies/maryland","us_agencies/massachusetts","us_agencies/michigan","us_agencies/minnesota","us_agencies/mississippi","us_agencies/missouri","us_agencies/montana","us_agencies/nebraska","us_agencies/nevada","us_agencies/newhampshire","us_agencies/newjersey","us_agencies/newmexico","us_agencies/newyork","us_agencies/northcarolina","us_agencies/northdakota","us_agencies/ohio","us_agencies/oklahoma","us_agencies/oregon","us_agencies/pennsylvania","us_agencies/rhodeisland","us_agencies/southcarolina","us_agencies/southdakota","us_agencies/tennessee","us_agencies/texas","us_agencies/utah","us_agencies/vermont","us_agencies/virginia","us_agencies/washington","us_agencies/westvirginia","us_agencies/wisconsin","us_agencies/wyoming","us_agencies/aggregate",
#            "eu_procurements_alt/AT_2008","eu_procurements_alt/AT_2009","eu_procurements_alt/AT_2010","eu_procurements_alt/AT_2011","eu_procurements_alt/AT_2012","eu_procurements_alt/AT_2013","eu_procurements_alt/AT_2014","eu_procurements_alt/AT_2015","eu_procurements_alt/AT_2016","eu_procurements_alt/BE_2008","eu_procurements_alt/BE_2009","eu_procurements_alt/BE_2010","eu_procurements_alt/BE_2011","eu_procurements_alt/BE_2012","eu_procurements_alt/BE_2013","eu_procurements_alt/BE_2014","eu_procurements_alt/BE_2015","eu_procurements_alt/BE_2016","eu_procurements_alt/BG_2008","eu_procurements_alt/BG_2009","eu_procurements_alt/BG_2010","eu_procurements_alt/BG_2011","eu_procurements_alt/BG_2012","eu_procurements_alt/BG_2013","eu_procurements_alt/BG_2014","eu_procurements_alt/BG_2015","eu_procurements_alt/BG_2016","eu_procurements_alt/CY_2008","eu_procurements_alt/CY_2009","eu_procurements_alt/CY_2010","eu_procurements_alt/CY_2011","eu_procurements_alt/CY_2012","eu_procurements_alt/CY_2013","eu_procurements_alt/CY_2014","eu_procurements_alt/CY_2015","eu_procurements_alt/CY_2016","eu_procurements_alt/CZ_2008","eu_procurements_alt/CZ_2009","eu_procurements_alt/CZ_2010","eu_procurements_alt/CZ_2011","eu_procurements_alt/CZ_2012","eu_procurements_alt/CZ_2013","eu_procurements_alt/CZ_2014","eu_procurements_alt/CZ_2015","eu_procurements_alt/CZ_2016","eu_procurements_alt/DE_2008","eu_procurements_alt/DE_2009","eu_procurements_alt/DE_2010","eu_procurements_alt/DE_2011","eu_procurements_alt/DE_2012","eu_procurements_alt/DE_2013","eu_procurements_alt/DE_2014","eu_procurements_alt/DE_2015","eu_procurements_alt/DE_2016","eu_procurements_alt/DK_2008","eu_procurements_alt/DK_2009","eu_procurements_alt/DK_2010","eu_procurements_alt/DK_2011","eu_procurements_alt/DK_2012","eu_procurements_alt/DK_2013","eu_procurements_alt/DK_2014","eu_procurements_alt/DK_2015","eu_procurements_alt/DK_2016","eu_procurements_alt/EE_2008","eu_procurements_alt/EE_2009","eu_procurements_alt/EE_2010","eu_procurements_alt/EE_2011","eu_procurements_alt/EE_2012","eu_procurements_alt/EE_2013","eu_procurements_alt/EE_2014","eu_procurements_alt/EE_2015","eu_procurements_alt/EE_2016","eu_procurements_alt/ES_2008","eu_procurements_alt/ES_2009","eu_procurements_alt/ES_2010","eu_procurements_alt/ES_2011","eu_procurements_alt/ES_2012","eu_procurements_alt/ES_2013","eu_procurements_alt/ES_2014","eu_procurements_alt/ES_2015","eu_procurements_alt/ES_2016","eu_procurements_alt/FI_2008","eu_procurements_alt/FI_2009","eu_procurements_alt/FI_2010","eu_procurements_alt/FI_2011","eu_procurements_alt/FI_2012","eu_procurements_alt/FI_2013","eu_procurements_alt/FI_2014","eu_procurements_alt/FI_2015","eu_procurements_alt/FI_2016","eu_procurements_alt/FR_2008","eu_procurements_alt/FR_2009","eu_procurements_alt/FR_2010","eu_procurements_alt/FR_2011","eu_procurements_alt/FR_2012","eu_procurements_alt/FR_2013","eu_procurements_alt/FR_2014","eu_procurements_alt/FR_2015","eu_procurements_alt/FR_2016","eu_procurements_alt/GR_2008","eu_procurements_alt/GR_2009","eu_procurements_alt/GR_2010","eu_procurements_alt/GR_2011","eu_procurements_alt/GR_2012","eu_procurements_alt/GR_2013","eu_procurements_alt/GR_2014","eu_procurements_alt/GR_2015","eu_procurements_alt/GR_2016","eu_procurements_alt/HU_2008","eu_procurements_alt/HU_2009","eu_procurements_alt/HU_2010","eu_procurements_alt/HU_2011","eu_procurements_alt/HU_2012","eu_procurements_alt/HU_2013","eu_procurements_alt/HU_2014","eu_procurements_alt/HU_2015","eu_procurements_alt/HU_2016","eu_procurements_alt/IE_2008","eu_procurements_alt/IE_2009","eu_procurements_alt/IE_2010","eu_procurements_alt/IE_2011","eu_procurements_alt/IE_2012","eu_procurements_alt/IE_2013","eu_procurements_alt/IE_2014","eu_procurements_alt/IE_2015","eu_procurements_alt/IE_2016","eu_procurements_alt/IT_2008","eu_procurements_alt/IT_2009","eu_procurements_alt/IT_2010","eu_procurements_alt/IT_2011","eu_procurements_alt/IT_2012","eu_procurements_alt/IT_2013","eu_procurements_alt/IT_2014","eu_procurements_alt/IT_2015","eu_procurements_alt/IT_2016","eu_procurements_alt/LT_2008","eu_procurements_alt/LT_2009","eu_procurements_alt/LT_2010","eu_procurements_alt/LT_2011","eu_procurements_alt/LT_2012","eu_procurements_alt/LT_2013","eu_procurements_alt/LT_2014","eu_procurements_alt/LT_2015","eu_procurements_alt/LT_2016","eu_procurements_alt/LV_2008","eu_procurements_alt/LV_2009","eu_procurements_alt/LV_2010","eu_procurements_alt/LV_2011","eu_procurements_alt/LV_2012","eu_procurements_alt/LV_2013","eu_procurements_alt/LV_2014","eu_procurements_alt/LV_2015","eu_procurements_alt/LV_2016","eu_procurements_alt/NL_2008","eu_procurements_alt/NL_2009","eu_procurements_alt/NL_2010","eu_procurements_alt/NL_2011","eu_procurements_alt/NL_2012","eu_procurements_alt/NL_2013","eu_procurements_alt/NL_2014","eu_procurements_alt/NL_2015","eu_procurements_alt/NL_2016","eu_procurements_alt/NO_2008","eu_procurements_alt/NO_2009","eu_procurements_alt/NO_2010","eu_procurements_alt/NO_2011","eu_procurements_alt/NO_2012","eu_procurements_alt/NO_2013","eu_procurements_alt/NO_2014","eu_procurements_alt/NO_2015","eu_procurements_alt/NO_2016","eu_procurements_alt/PL_2008","eu_procurements_alt/PL_2009","eu_procurements_alt/PL_2010","eu_procurements_alt/PL_2011","eu_procurements_alt/PL_2012","eu_procurements_alt/PL_2013","eu_procurements_alt/PL_2014","eu_procurements_alt/PL_2015","eu_procurements_alt/PL_2016","eu_procurements_alt/PT_2008","eu_procurements_alt/PT_2009","eu_procurements_alt/PT_2010","eu_procurements_alt/PT_2011","eu_procurements_alt/PT_2012","eu_procurements_alt/PT_2013","eu_procurements_alt/PT_2014","eu_procurements_alt/PT_2015","eu_procurements_alt/PT_2016","eu_procurements_alt/RO_2008","eu_procurements_alt/RO_2009","eu_procurements_alt/RO_2010","eu_procurements_alt/RO_2011","eu_procurements_alt/RO_2012","eu_procurements_alt/RO_2013","eu_procurements_alt/RO_2014","eu_procurements_alt/RO_2015","eu_procurements_alt/RO_2016","eu_procurements_alt/SE_2008","eu_procurements_alt/SE_2009","eu_procurements_alt/SE_2010","eu_procurements_alt/SE_2011","eu_procurements_alt/SE_2012","eu_procurements_alt/SE_2013","eu_procurements_alt/SE_2014","eu_procurements_alt/SE_2015","eu_procurements_alt/SE_2016","eu_procurements_alt/SI_2008","eu_procurements_alt/SI_2009","eu_procurements_alt/SI_2010","eu_procurements_alt/SI_2011","eu_procurements_alt/SI_2012","eu_procurements_alt/SI_2013","eu_procurements_alt/SI_2014","eu_procurements_alt/SI_2015","eu_procurements_alt/SI_2016","eu_procurements_alt/SK_2008","eu_procurements_alt/SK_2009","eu_procurements_alt/SK_2010","eu_procurements_alt/SK_2011","eu_procurements_alt/SK_2012","eu_procurements_alt/SK_2013","eu_procurements_alt/SK_2014","eu_procurements_alt/SK_2015","eu_procurements_alt/SK_2016","eu_procurements_alt/UK_2008","eu_procurements_alt/UK_2009","eu_procurements_alt/UK_2010","eu_procurements_alt/UK_2011","eu_procurements_alt/UK_2012","eu_procurements_alt/UK_2013","eu_procurements_alt/UK_2014","eu_procurements_alt/UK_2015","eu_procurements_alt/UK_2016"
#             "jester","libimseti","wiki_article_words","bookcrossing",
#             "twitter_higgs/social","twitter_higgs/retweet","twitter_higgs/reply","twitter_higgs/mention","twitter_higgs/activity",
#             "eu_procurements"  ,"epinions",
#             #"yahoo_song", #DOES NOT WORK ON COLAB: TOO MUCH RAM
#              "mag_history_coauthor/max-25-bip","mag_history_coauthor/max-25-proj","mag_history_coauthor/full-bip","mag_history_coauthor/full-proj",
#              "mag_geology_coauthor/max-25-bip","mag_geology_coauthor/max-25-proj","mag_geology_coauthor/full-bip","mag_geology_coauthor/full-proj",
#              "amazon_ratings",
#              "dblp_simplices/max-25-bip","dblp_simplices/max-25-proj","dblp_simplices/full-bip","dblp_simplices/full-proj",
#              "bag_of_words/enron","bag_of_words/kos","bag_of_words/nips","bag_of_words/nytimes", #"bag_of_words/pubmed", TOO BIG
#              "microsoft_concept"
#]

#Unfortunately, not all models classified as weighted have weights. We handle these cases setting deafult weight 1
#model_names=["hens"] # DOES NOT HAVE WEIGHTS! The default '1' will be set to all edges

prefix_to_weight={'eu_procurements_alt/':'pctSingleBid','arxiv_collab/':'value','bag_of_words/':'count',
                  'budapest_connectome/':'occurences','celegans_2019':'connectivity',
                  'us_agencies/':'link_counts','plant_pol_vazquez':'count'}

#Set flags on where to store the obtained CSV files
download_file=False
store_on_gdrive=True
path_g_drive="weightedGraphs/"
export_all_weightedGraphs_erodeCSV(model_names,download_file,store_on_gdrive,path_g_drive,prefix_to_weight)

ModuleNotFoundError: No module named 'pandas'

# If interested in drawing

In [ ]:
# from graph_tool.all import *
# g = collection.data["celegansneural"]
# state = minimize_nested_blockmodel_dl(g)
# state = minimize_nested_blockmodel_dl(g)
# state.draw()